In [15]:
import json
from typing import List, Union
from pathlib import Path
import re

import pandas as pd
import requests
import yaml

In [16]:
CONFIG_FILENAME = "/home/jovyan/work/mobilidade-rio-api/mobilidade_rio/helpers/config_gtfs.yaml"#"config_gtfs.yaml"

In [17]:
#supressão de serviço tipo SN
#Tipo de serviço é o 8 character do id de rotas.
er=[]
def is_not_SN(short_name):
    #print(route_id,route_id[7],route_id[7]=="N")
    #print(short_name, re.search("^SN", short_name))
    try:
        return re.search("^SN", short_name)
    
    except:
        #print("error ", route_id)
        er.append(route_id)
        return True

is_not_SN("O0624AAN0A")



In [18]:
def load_csv(filepath: Union[str, Path]) -> pd.DataFrame:
    """
    Carrega o arquivo CSV e retorna um DataFrame.
    """
    return pd.read_csv(filepath)

def dump_fixtures(fixtures: List[dict], filepath: Union[str, Path]):
    """
    Salva os fixtures em um arquivo JSON.
    """
    with open(filepath, "w") as f:
        json.dump(fixtures, f, indent=4)

def resolve_e(case,record,i):
    if case == 'trip_id[12:15]':
        return record['trip_id'][12:15]
    if case =='i':
        return str(i)
    if case == "": #TODO: resolver o caso da via.
        return ""
    if case == "stop_id[4:5]":
        return record["stop_id"][4:5]
    if case in ['3',3]:#set modal hard coded.
        return int(3)
    
#usado aqui para controlar a execução de fixtures sem alterar arquivos.
a={}
def gtfs_to_fixtures(dataframe, config_json, model, trip_list):
    fixtures = [] 
    k=config_json["json"]["fields"].keys()
    print(k)
    e=['trip_id[12:15]', "i","","stop_id[4:5]",3,"3"]
    for i, record in dataframe.iterrows():
        result = {
                "model": "pontos."+config_json["json"]["model"],
                "pk": record[config_json["json"]["pk"]] if config_json["json"]["pk"] not in e else resolve_e(config_json["json"]["pk"],record,i),
                "fields": dict.fromkeys(k,"")
            }
        
        #print(result)
        for item in k:
            if config_json["json"]["fields"][item] not in e:
                result["fields"][item]=record[config_json["json"]["fields"][item]]
            else:
                result["fields"][item]=resolve_e(config_json["json"]["fields"][item],record,i)
        
            if model =="agency":
                result["pk"]=str(result["pk"])
            if model =="sequence":
                result["pk"]=int(result["pk"])
            if model =="route":
                result["fields"]['agency']=str(result["fields"]['agency'])
            if model =="trip":
                if result["fields"]['headsign']!=result["fields"]['headsign']:#check for NaN
                    result["fields"]['headsign']=""
                
            if model =="stop":
                result["fields"]['latitude']=str(result["fields"]['latitude'])
                result["fields"]['longitude']=str(result["fields"]['longitude'])

        if model == "sequence" and result["fields"]["trip"] in trip_list:# retira trips que são SNs
            fixtures.append(result)
        if model not in ["sequence"]:
            fixtures.append(result)
    
    dump_fixtures(fixtures, f"/home/jovyan/work/smtr/mobilidade-rio-api/mobilidade_rio/fixtures/{model}.json")
    display(f"Adicionado {len(fixtures)} a {model}")

In [35]:
# casos especiais:

ID_MODAL={
    'MM':'M',
    '10':'M',
    '11':'T',
    '12':'V',
    '13':'S',
    '14':'P',
    '20':'B',
    '21':'H', 
    '22':'O',
    '23':'E',
    '30':'I',
    '31 a 39':'I',
    '----':'J',
    '40':'L',
    '41':'C',
    '50':'F',
    '60':'A'
}

def translate_to_new_mode(mode):
    ''''
    if mode!='MM' and int(mode)>30 and int(mode)<39:
        return "I"
    if str(mode) not in ID_MODAL:
        return "J"
    else:
        return ID_MODAL[str(mode)]
    '''
    return 3

ID_CORREDOR={
    '0':'0',
    '1':'O',
    '2':'M',
    '3':'C',
    '4':'L',
    '5':'B',
    '6':'Z',
    '7':'Y'
}

def translate_to_new_corredor(corredor):
    return ID_CORREDOR[corredor]


TIPO_PARADA={
    '0':'0',
    '1':'C',
    '2':'A',
    '4':'P',
    '5':'E',
    '6':'S',
    '7':'U',
    '8':'X',
    '9':'T'
}
def translate_to_new_parada(parada):
    return TIPO_PARADA[parada]

print('dicionarios de para geração dos ids preparados')

def treat_results(stop,code,count,i):
    return {
            "model": "pontos.qrcode",
            "pk": count+i,
            "fields": {
                "stop": stop,
                "code": code,
            }
    }

dicionarios de para geração dos ids preparados


In [36]:
config = yaml.load(
        open(CONFIG_FILENAME, "r"), 
        Loader=yaml.FullLoader
    )

config["models"].keys()

dict_keys(['agency', 'linha', 'route', 'trip', 'stop', 'sequence'])

In [37]:
trip_list=[]
#usado para gerar todos os arquivos de fixture descritos em config.yml 
#trip precisa ser gerando antes de sequence 
for model in config["models"].keys():
    config_json=config["models"][model]
    dataframe = pd.read_csv( config_json['source'] )
    if model == "sequence":
        trip_list=pd.json_normalize(json.load(open("/home/jovyan/work/smtr/mobilidade-rio-api/mobilidade_rio/fixtures/trip.json")))['pk'].tolist()
        dataframe["trip_id"]= dataframe["trip_id"].str.split("_").str[0]
        dataframe.drop_duplicates(subset=['trip_id', 'stop_id'], keep="first", inplace=True)
        
    if model == "trip":
        trip_list=[]
        route_list=pd.json_normalize(json.load(open("/home/jovyan/work/smtr/mobilidade-rio-api/mobilidade_rio/fixtures/route.json")))['pk'].tolist()
        dataframe=dataframe.loc[dataframe["route_id"].isin(route_list)]
        dataframe["trip_id"]= dataframe["trip_id"].str.split("_").str[0]
        dataframe.drop_duplicates(subset='trip_id', keep="first", inplace=True)
        
    if model in ["route","linha"]:
        dataframe=dataframe.loc[dataframe["route_short_name"].str.contains("(SN)")==False]
        
    display(f"{model}  {dataframe.shape[0]}")
    gtfs_to_fixtures(dataframe, config_json, model,trip_list)

'agency  4'

dict_keys(['name'])


'Adicionado 4 a agency'

/tmp/ipykernel_74/2591813014.py:23: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dataframe=dataframe.loc[dataframe["route_short_name"].str.contains("(SN)")==False]


'linha  365'

...

'Adicionado 6710 a stop'

/tmp/ipykernel_74/2591813014.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv( config_json['source'] )


'sequence  43565'

dict_keys(['trip', 'stop', 'order'])


'Adicionado 37846 a sequence'

In [7]:
# generate qr-code.

In [8]:
# Generate from api

In [57]:
import requests
last_page =True
#url='https://api.mobilidade.rio/qrcode/'
url="https://api.staging.mobilidade.rio/qrcode/"
fixtures=[]

while last_page:
    x = requests.get(url)
    x=json.loads(x.text)
    results=x['results']
    for item in results:
        code = item["code"]
        stop=item['stop']["id"]
        if len(stop)>12:
            stop=convert_old_to_new_stop(stop)
        fixtures.append(treat_results(stop,code,0,i))

    if x["next"]:
        url= x["next"]
    else:
        last_page=False

print(len(fixtures),'pontos adicionados a fixtures')

44 pontos adicionados a fixtures


In [74]:
fixtures=[]

In [58]:
new_code=[
    ["CZ9H","t2u9"],
    ["DQ2D","3061O00018C0"],
    ["T5TT","3061O00019C0"],
    ["HFGL","3061O00020C0"],
    ["DRVV","3061O00021C0"],
    ["TDBA","3061O00022C0"],
    ["DQ3X","3061O00023C0"]
]
for i in range(len(new_code)):
        code = new_code[i][0]
        stop = new_code[i][1]
        if len(stop)>12:
            stop=convert_old_to_new_stop(stop)
        fixtures.append(treat_results(stop,code,len(fixtures),0))
print(len(fixtures),'pontos adicionados a fixtures')

51 pontos adicionados a fixtures


In [14]:
#create json

In [59]:
dump_fixtures(fixtures, f"../fixtures/qrcode.json")

In [161]:
#generate mode.json from api

In [73]:
import requests

url='https://api.mobilidade.rio/mode/'
fixtures=[]
x = requests.get(url)
x=json.loads(x.text)
results=x['results']
for item in results:
    modal={
            "model": "pontos.mode",
            "pk": translate_to_new_mode(item["id"]),
            "fields": {
                "name": item["name"],
            }
                
    }
    fixtures.append(modal)

print(len(fixtures),'modais adicionados a fixtures')
dump_fixtures(fixtures, f"../fixtures/mode.json")

11 modais adicionados a fixtures
